# Multi-Task Neural Network (MTNN) for Typhoon Impact Prediction

## OPTION B: Multi-Task Learning with Shared Layers and Multiple Output Heads

This notebook implements a Multi-Task Learning (MTL) neural network that uses:
- **Shared dense layers** for feature extraction
- **Multiple output heads** for different prediction tasks:
  - **Regression heads**: families, person, brgy, cost, partially, totally
  - **Classification heads**: dead, injured_ill, missing (binary labels)

### Benefits of Multi-Task Learning:
1. Shared representations improve generalization
2. Regularization effect from auxiliary tasks
3. Efficient parameter sharing
4. Better handling of rare events through joint learning

## 1. Import Required Libraries

In [2]:
%pip install tensorflow

  Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl (12.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Rodney Lei Estrada\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\~umpy\\.libs\\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# Core Libraries
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Metrics
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    accuracy_score, f1_score, roc_auc_score, confusion_matrix,
    classification_report
)

# TensorFlow/Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

ImportError: Traceback (most recent call last):
  File "C:\Users\Rodney Lei Estrada\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

## 2. Load and Explore Dataset

In [ ]:
# Load the dataset
data_path = '../data/typhoon_impact_with_extreme_weather.csv'
df = pd.read_csv(data_path)

# Display basic information
print("Dataset Shape:", df.shape)
print("\nColumn Names:")
print(df.columns.tolist())
print("\nFirst 5 rows:")
df.head()

In [ ]:
# Check data types and missing values
print("Data Types:")
print(df.dtypes)
print("\nMissing Values:")
print(df.isnull().sum())

## 3. Data Preprocessing

In [ ]:
# Define input features and target columns
INPUT_FEATURES = [
    'Max_Sustained_Wind_kph',
    'Typhoon_Type',
    'Max_24hr_Rainfall_mm',
    'Total_Storm_Rainfall_mm',
    'Min_Pressure_hPa'
]

# Regression targets
REGRESSION_TARGETS = ['Families', 'Person', 'Brgy', 'Cost', 'Partially', 'Totally']

# Classification targets (will be converted to binary)
CLASSIFICATION_TARGETS = ['Dead', 'Injured/Ill', 'Missing']

# Clean column names for output (lowercase, no special chars)
REGRESSION_OUTPUT_NAMES = ['families', 'person', 'brgy', 'cost', 'partially', 'totally']
CLASSIFICATION_OUTPUT_NAMES = ['dead', 'injured_ill', 'missing']

print("Input Features:", INPUT_FEATURES)
print("\nRegression Targets:", REGRESSION_TARGETS)
print("\nClassification Targets:", CLASSIFICATION_TARGETS)

In [ ]:
# Create a working copy of the dataframe
df_model = df.copy()

# Handle missing values in input features
for col in INPUT_FEATURES:
    if col in df_model.columns:
        if df_model[col].dtype == 'object':
            df_model[col] = df_model[col].fillna(df_model[col].mode()[0])
        else:
            df_model[col] = df_model[col].fillna(df_model[col].median())

# Handle missing values in target columns
for col in REGRESSION_TARGETS + CLASSIFICATION_TARGETS:
    if col in df_model.columns:
        df_model[col] = df_model[col].fillna(0)

print("Missing values after imputation:")
print(df_model[INPUT_FEATURES + REGRESSION_TARGETS + CLASSIFICATION_TARGETS].isnull().sum())

In [ ]:
# Encode categorical feature: Typhoon_Type
label_encoder = LabelEncoder()
df_model['Typhoon_Type_Encoded'] = label_encoder.fit_transform(df_model['Typhoon_Type'])

# Display typhoon type mapping
typhoon_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Typhoon Type Encoding:")
for k, v in typhoon_mapping.items():
    print(f"  {k}: {v}")

In [ ]:
# Create binary classification labels (value > 0)
df_model['Dead_Binary'] = (df_model['Dead'] > 0).astype(int)
df_model['Injured_Ill_Binary'] = (df_model['Injured/Ill'] > 0).astype(int)
df_model['Missing_Binary'] = (df_model['Missing'] > 0).astype(int)

# Check class distribution for binary targets
print("Binary Classification Label Distribution:")
print(f"\nDead (>0): {df_model['Dead_Binary'].sum()} / {len(df_model)} ({df_model['Dead_Binary'].mean()*100:.2f}%)")
print(f"Injured/Ill (>0): {df_model['Injured_Ill_Binary'].sum()} / {len(df_model)} ({df_model['Injured_Ill_Binary'].mean()*100:.2f}%)")
print(f"Missing (>0): {df_model['Missing_Binary'].sum()} / {len(df_model)} ({df_model['Missing_Binary'].mean()*100:.2f}%)")

In [ ]:
# Prepare feature matrix X
feature_columns = [
    'Max_Sustained_Wind_kph',
    'Typhoon_Type_Encoded',
    'Max_24hr_Rainfall_mm',
    'Total_Storm_Rainfall_mm',
    'Min_Pressure_hPa'
]

X = df_model[feature_columns].values

# Prepare target dictionaries
# Regression targets
y_regression = {
    'families': df_model['Families'].values,
    'person': df_model['Person'].values,
    'brgy': df_model['Brgy'].values,
    'cost': df_model['Cost'].values,
    'partially': df_model['Partially'].values,
    'totally': df_model['Totally'].values
}

# Classification targets (binary)
y_classification = {
    'dead': df_model['Dead_Binary'].values,
    'injured_ill': df_model['Injured_Ill_Binary'].values,
    'missing': df_model['Missing_Binary'].values
}

print(f"Feature matrix shape: {X.shape}")
print(f"Number of regression targets: {len(y_regression)}")
print(f"Number of classification targets: {len(y_classification)}")

## 4. Train/Test Split and Feature Scaling

In [ ]:
# Combine all targets for splitting
y_all = {**y_regression, **y_classification}

# Create indices for splitting
indices = np.arange(len(X))
train_idx, test_idx = train_test_split(indices, test_size=0.2, random_state=42)

# Split features
X_train = X[train_idx]
X_test = X[test_idx]

# Split targets
y_train = {key: val[train_idx] for key, val in y_all.items()}
y_test = {key: val[test_idx] for key, val in y_all.items()}

print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

In [ ]:
# Scale input features
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# Scale regression targets for better training
scalers_y = {}
y_train_scaled = {}
y_test_scaled = {}

for target in REGRESSION_OUTPUT_NAMES:
    scaler = StandardScaler()
    y_train_scaled[target] = scaler.fit_transform(y_train[target].reshape(-1, 1)).flatten()
    y_test_scaled[target] = scaler.transform(y_test[target].reshape(-1, 1)).flatten()
    scalers_y[target] = scaler

# Classification targets don't need scaling
for target in CLASSIFICATION_OUTPUT_NAMES:
    y_train_scaled[target] = y_train[target]
    y_test_scaled[target] = y_test[target]

print("Feature and target scaling complete.")
print(f"\nScaled X_train stats: mean={X_train_scaled.mean():.4f}, std={X_train_scaled.std():.4f}")

## 5. Build Multi-Task Neural Network Model

In [ ]:
def build_mtl_model(input_dim, regression_targets, classification_targets):
    """
    Build a Multi-Task Learning model with shared layers and multiple output heads.
    
    Architecture:
    - Input Layer
    - Shared Dense Layers (128 -> 64 -> 32)
    - Separate output heads for each target
    
    Parameters:
    -----------
    input_dim : int
        Number of input features
    regression_targets : list
        List of regression target names
    classification_targets : list
        List of classification target names
    
    Returns:
    --------
    model : keras.Model
        Compiled MTL model
    """
    
    # Input layer
    inputs = Input(shape=(input_dim,), name='input')
    
    # Shared layers
    shared = layers.Dense(128, activation='relu', name='shared_dense_1')(inputs)
    shared = layers.BatchNormalization(name='shared_bn_1')(shared)
    shared = layers.Dropout(0.3, name='shared_dropout_1')(shared)
    
    shared = layers.Dense(64, activation='relu', name='shared_dense_2')(shared)
    shared = layers.BatchNormalization(name='shared_bn_2')(shared)
    shared = layers.Dropout(0.2, name='shared_dropout_2')(shared)
    
    shared = layers.Dense(32, activation='relu', name='shared_dense_3')(shared)
    shared = layers.BatchNormalization(name='shared_bn_3')(shared)
    shared = layers.Dropout(0.1, name='shared_dropout_3')(shared)
    
    # Output heads dictionary
    outputs = {}
    
    # Regression output heads
    for target in regression_targets:
        # Task-specific layer
        task_layer = layers.Dense(16, activation='relu', name=f'{target}_dense')(shared)
        # Output: linear activation for regression
        outputs[target] = layers.Dense(1, activation='linear', name=target)(task_layer)
    
    # Classification output heads
    for target in classification_targets:
        # Task-specific layer
        task_layer = layers.Dense(16, activation='relu', name=f'{target}_dense')(shared)
        # Output: sigmoid activation for binary classification
        outputs[target] = layers.Dense(1, activation='sigmoid', name=target)(task_layer)
    
    # Create model
    model = Model(inputs=inputs, outputs=outputs, name='MTL_Model')
    
    return model

# Build the model
model = build_mtl_model(
    input_dim=X_train_scaled.shape[1],
    regression_targets=REGRESSION_OUTPUT_NAMES,
    classification_targets=CLASSIFICATION_OUTPUT_NAMES
)

# Display model summary
model.summary()

In [ ]:
# Visualize model architecture
try:
    tf.keras.utils.plot_model(
        model,
        to_file='../models/mtl_model_architecture.png',
        show_shapes=True,
        show_layer_names=True,
        rankdir='TB',
        dpi=100
    )
    print("Model architecture saved to '../models/mtl_model_architecture.png'")
except Exception as e:
    print(f"Could not save model architecture image: {e}")

## 6. Define Loss Functions and Compile Model

In [ ]:
# Define losses for each output head
losses = {
    # Regression losses (MSE)
    'families': 'mse',
    'person': 'mse',
    'brgy': 'mse',
    'cost': 'mse',
    'partially': 'mse',
    'totally': 'mse',
    # Classification losses (Binary Cross-Entropy)
    'dead': 'binary_crossentropy',
    'injured_ill': 'binary_crossentropy',
    'missing': 'binary_crossentropy'
}

# Define loss weights to balance regression vs classification
# Classification tasks are given higher weight due to class imbalance
loss_weights = {
    # Regression weights
    'families': 1.0,
    'person': 1.0,
    'brgy': 1.0,
    'cost': 0.5,  # Cost can be very large, reduce weight
    'partially': 1.0,
    'totally': 1.0,
    # Classification weights (higher for rare events)
    'dead': 5.0,
    'injured_ill': 3.0,
    'missing': 4.0
}

# Define metrics for each output
metrics = {
    'families': ['mae'],
    'person': ['mae'],
    'brgy': ['mae'],
    'cost': ['mae'],
    'partially': ['mae'],
    'totally': ['mae'],
    'dead': ['accuracy', 'AUC'],
    'injured_ill': ['accuracy', 'AUC'],
    'missing': ['accuracy', 'AUC']
}

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=losses,
    loss_weights=loss_weights,
    metrics=metrics
)

print("Model compiled successfully!")
print("\nLoss functions:")
for k, v in losses.items():
    print(f"  {k}: {v} (weight: {loss_weights[k]})")

## 7. Train the MTL Model

In [ ]:
# Define callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
        verbose=1
    )
]

# Train the model
print("Training Multi-Task Learning Model...")
print("="*50)

history = model.fit(
    X_train_scaled,
    y_train_scaled,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

print("\nTraining complete!")

## 8. Visualize Training History

In [ ]:
def plot_training_history(history):
    """
    Plot training and validation loss curves.
    """
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Plot total loss
    ax = axes[0, 0]
    ax.plot(history.history['loss'], label='Training Loss', linewidth=2)
    ax.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
    ax.set_title('Total Loss', fontsize=12, fontweight='bold')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot regression losses
    ax = axes[0, 1]
    for target in REGRESSION_OUTPUT_NAMES:
        if f'{target}_loss' in history.history:
            ax.plot(history.history[f'{target}_loss'], label=f'{target}', alpha=0.7)
    ax.set_title('Regression Losses (Training)', fontsize=12, fontweight='bold')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss (MSE)')
    ax.legend(loc='upper right', fontsize=8)
    ax.grid(True, alpha=0.3)
    
    # Plot classification losses
    ax = axes[1, 0]
    for target in CLASSIFICATION_OUTPUT_NAMES:
        if f'{target}_loss' in history.history:
            ax.plot(history.history[f'{target}_loss'], label=f'{target}', linewidth=2)
    ax.set_title('Classification Losses (Training)', fontsize=12, fontweight='bold')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss (BCE)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot classification AUC
    ax = axes[1, 1]
    for target in CLASSIFICATION_OUTPUT_NAMES:
        auc_key = f'{target}_auc'
        # Handle different naming conventions in Keras
        for key in history.history.keys():
            if target in key.lower() and 'auc' in key.lower() and 'val' not in key.lower():
                ax.plot(history.history[key], label=f'{target} (train)', linewidth=2)
                break
    ax.set_title('Classification AUC (Training)', fontsize=12, fontweight='bold')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('AUC')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../models/mtl_training_history.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("Training history plot saved to '../models/mtl_training_history.png'")

plot_training_history(history)

## 9. Model Evaluation

In [ ]:
# Get predictions on test set
predictions = model.predict(X_test_scaled)

# Convert predictions to dictionary if not already
if isinstance(predictions, list):
    output_names = REGRESSION_OUTPUT_NAMES + CLASSIFICATION_OUTPUT_NAMES
    predictions = {name: pred.flatten() for name, pred in zip(output_names, predictions)}

print("Predictions generated for test set.")

### 9.1 Regression Metrics

In [ ]:
def evaluate_regression(y_true, y_pred, target_name, scaler=None):
    """
    Evaluate regression performance.
    
    Returns:
    --------
    dict : Dictionary containing MAE, RMSE, and R² scores
    """
    # Inverse transform if scaler provided
    if scaler is not None:
        y_true_orig = scaler.inverse_transform(y_true.reshape(-1, 1)).flatten()
        y_pred_orig = scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()
    else:
        y_true_orig = y_true
        y_pred_orig = y_pred
    
    mae = mean_absolute_error(y_true_orig, y_pred_orig)
    rmse = np.sqrt(mean_squared_error(y_true_orig, y_pred_orig))
    r2 = r2_score(y_true_orig, y_pred_orig)
    
    return {
        'Target': target_name,
        'MAE': mae,
        'RMSE': rmse,
        'R²': r2
    }, y_true_orig, y_pred_orig

# Evaluate all regression targets
regression_results = []
regression_predictions = {}

print("Regression Evaluation Results")
print("=" * 60)

for target in REGRESSION_OUTPUT_NAMES:
    y_true = y_test_scaled[target]
    y_pred = predictions[target].flatten()
    
    result, y_true_orig, y_pred_orig = evaluate_regression(
        y_true, y_pred, target, scalers_y.get(target)
    )
    regression_results.append(result)
    regression_predictions[target] = {'true': y_true_orig, 'pred': y_pred_orig}

# Display results as DataFrame
df_regression_results = pd.DataFrame(regression_results)
print(df_regression_results.to_string(index=False))

### 9.2 Classification Metrics

In [ ]:
def evaluate_classification(y_true, y_pred_proba, target_name, threshold=0.5):
    """
    Evaluate classification performance.
    
    Returns:
    --------
    dict : Dictionary containing Accuracy, F1, and ROC-AUC scores
    """
    y_pred = (y_pred_proba >= threshold).astype(int)
    
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    
    # Handle cases where only one class is present
    try:
        roc_auc = roc_auc_score(y_true, y_pred_proba)
    except ValueError:
        roc_auc = np.nan
    
    return {
        'Target': target_name,
        'Accuracy': accuracy,
        'F1 Score': f1,
        'ROC-AUC': roc_auc
    }, y_pred

# Evaluate all classification targets
classification_results = []
classification_predictions = {}

print("\nClassification Evaluation Results")
print("=" * 60)

for target in CLASSIFICATION_OUTPUT_NAMES:
    y_true = y_test_scaled[target]
    y_pred_proba = predictions[target].flatten()
    
    result, y_pred = evaluate_classification(y_true, y_pred_proba, target)
    classification_results.append(result)
    classification_predictions[target] = {
        'true': y_true,
        'pred': y_pred,
        'proba': y_pred_proba
    }

# Display results as DataFrame
df_classification_results = pd.DataFrame(classification_results)
print(df_classification_results.to_string(index=False))

## 10. Visualization: Actual vs Predicted (Regression)

In [ ]:
def plot_actual_vs_predicted(regression_predictions, save_path=None):
    """
    Create actual vs predicted scatter plots for regression targets.
    """
    n_targets = len(regression_predictions)
    n_cols = 3
    n_rows = (n_targets + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
    axes = axes.flatten()
    
    for idx, (target, data) in enumerate(regression_predictions.items()):
        ax = axes[idx]
        y_true = data['true']
        y_pred = data['pred']
        
        # Scatter plot
        ax.scatter(y_true, y_pred, alpha=0.5, edgecolors='none', s=30)
        
        # Perfect prediction line
        min_val = min(y_true.min(), y_pred.min())
        max_val = max(y_true.max(), y_pred.max())
        ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
        
        # Calculate R²
        r2 = r2_score(y_true, y_pred)
        
        ax.set_xlabel('Actual Values', fontsize=10)
        ax.set_ylabel('Predicted Values', fontsize=10)
        ax.set_title(f'{target.upper()}\n(R² = {r2:.4f})', fontsize=11, fontweight='bold')
        ax.legend(loc='upper left', fontsize=8)
        ax.grid(True, alpha=0.3)
    
    # Hide empty subplots
    for idx in range(n_targets, len(axes)):
        axes[idx].set_visible(False)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"Plot saved to '{save_path}'")
    
    plt.show()

# Plot actual vs predicted for regression targets
plot_actual_vs_predicted(
    regression_predictions,
    save_path='../models/mtl_actual_vs_predicted.png'
)

## 11. Visualization: Confusion Matrices (Classification)

In [ ]:
def plot_confusion_matrices(classification_predictions, save_path=None):
    """
    Create confusion matrix heatmaps for classification targets.
    """
    n_targets = len(classification_predictions)
    fig, axes = plt.subplots(1, n_targets, figsize=(5*n_targets, 4))
    
    if n_targets == 1:
        axes = [axes]
    
    for idx, (target, data) in enumerate(classification_predictions.items()):
        ax = axes[idx]
        y_true = data['true']
        y_pred = data['pred']
        
        # Compute confusion matrix
        cm = confusion_matrix(y_true, y_pred)
        
        # Plot heatmap
        sns.heatmap(
            cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Negative (0)', 'Positive (1)'],
            yticklabels=['Negative (0)', 'Positive (1)'],
            ax=ax, cbar=True
        )
        
        ax.set_xlabel('Predicted', fontsize=10)
        ax.set_ylabel('Actual', fontsize=10)
        ax.set_title(f'{target.upper()}\nConfusion Matrix', fontsize=11, fontweight='bold')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"Plot saved to '{save_path}'")
    
    plt.show()

# Plot confusion matrices
plot_confusion_matrices(
    classification_predictions,
    save_path='../models/mtl_confusion_matrices.png'
)

In [ ]:
# Print detailed classification reports
print("\nDetailed Classification Reports")
print("=" * 60)

for target in CLASSIFICATION_OUTPUT_NAMES:
    print(f"\n{target.upper()}:")
    print("-" * 40)
    y_true = classification_predictions[target]['true']
    y_pred = classification_predictions[target]['pred']
    print(classification_report(y_true, y_pred, target_names=['No Event', 'Event'], zero_division=0))

## 12. Summary of Results

In [ ]:
# Create a comprehensive summary
print("\n" + "="*70)
print("MULTI-TASK NEURAL NETWORK - EVALUATION SUMMARY")
print("="*70)

print("\n📊 REGRESSION PERFORMANCE:")
print("-"*50)
print(df_regression_results.to_string(index=False))

print("\n📈 CLASSIFICATION PERFORMANCE:")
print("-"*50)
print(df_classification_results.to_string(index=False))

# Calculate average metrics
avg_r2 = df_regression_results['R²'].mean()
avg_mae = df_regression_results['MAE'].mean()
avg_accuracy = df_classification_results['Accuracy'].mean()
avg_f1 = df_classification_results['F1 Score'].mean()
avg_auc = df_classification_results['ROC-AUC'].mean()

print("\n📋 AVERAGE METRICS:")
print("-"*50)
print(f"  Regression Average R²: {avg_r2:.4f}")
print(f"  Regression Average MAE: {avg_mae:.4f}")
print(f"  Classification Average Accuracy: {avg_accuracy:.4f}")
print(f"  Classification Average F1 Score: {avg_f1:.4f}")
print(f"  Classification Average ROC-AUC: {avg_auc:.4f}")
print("="*70)

## 13. Save the Trained Model

In [ ]:
import os
import joblib

# Create models directory if it doesn't exist
os.makedirs('../models', exist_ok=True)

# Save the model in H5 format
model_path = '../models/mtl_typhoon_prediction_model.h5'
model.save(model_path)
print(f"✅ Model saved to: {model_path}")

# Save the model in Keras format as well
model_keras_path = '../models/mtl_typhoon_prediction_model.keras'
model.save(model_keras_path)
print(f"✅ Model saved to: {model_keras_path}")

# Save scalers for future inference
scalers_path = '../models/mtl_scalers.joblib'
joblib.dump({
    'scaler_X': scaler_X,
    'scalers_y': scalers_y,
    'label_encoder': label_encoder
}, scalers_path)
print(f"✅ Scalers saved to: {scalers_path}")

# Save evaluation results
results_path = '../models/mtl_evaluation_results.joblib'
joblib.dump({
    'regression_results': df_regression_results,
    'classification_results': df_classification_results,
    'training_history': history.history
}, results_path)
print(f"✅ Evaluation results saved to: {results_path}")

## 14. Model Inference Example

In [ ]:
def predict_typhoon_impact(model, scaler_X, scalers_y, label_encoder, typhoon_data):
    """
    Make predictions for new typhoon data.
    
    Parameters:
    -----------
    model : keras.Model
        Trained MTL model
    scaler_X : StandardScaler
        Feature scaler
    scalers_y : dict
        Target scalers for regression outputs
    label_encoder : LabelEncoder
        Typhoon type encoder
    typhoon_data : dict
        Dictionary with typhoon features
    
    Returns:
    --------
    dict : Predictions for all targets
    """
    # Encode typhoon type
    typhoon_type_encoded = label_encoder.transform([typhoon_data['typhoon_type']])[0]
    
    # Create feature vector
    X_new = np.array([[
        typhoon_data['max_sustained_wind_kph'],
        typhoon_type_encoded,
        typhoon_data['max_24hr_rainfall_mm'],
        typhoon_data['total_storm_rainfall_mm'],
        typhoon_data['min_pressure_hpa']
    ]])
    
    # Scale features
    X_new_scaled = scaler_X.transform(X_new)
    
    # Get predictions
    predictions = model.predict(X_new_scaled, verbose=0)
    
    # Process predictions
    results = {}
    
    # Regression predictions (inverse transform)
    for i, target in enumerate(REGRESSION_OUTPUT_NAMES):
        pred_scaled = predictions[target][0][0]
        pred_original = scalers_y[target].inverse_transform([[pred_scaled]])[0][0]
        results[target] = max(0, pred_original)  # Ensure non-negative
    
    # Classification predictions
    for target in CLASSIFICATION_OUTPUT_NAMES:
        proba = predictions[target][0][0]
        results[f'{target}_probability'] = proba
        results[f'{target}_prediction'] = 'Yes' if proba >= 0.5 else 'No'
    
    return results

# Example prediction
example_typhoon = {
    'max_sustained_wind_kph': 195,
    'typhoon_type': 'STY',  # Super Typhoon
    'max_24hr_rainfall_mm': 300,
    'total_storm_rainfall_mm': 450,
    'min_pressure_hpa': 940
}

print("\n🌀 EXAMPLE PREDICTION")
print("="*50)
print("\nInput Typhoon Data:")
for k, v in example_typhoon.items():
    print(f"  {k}: {v}")

# Make prediction
prediction_results = predict_typhoon_impact(
    model, scaler_X, scalers_y, label_encoder, example_typhoon
)

print("\nPredicted Impact:")
print("-"*50)
print("\nRegression Predictions:")
for target in REGRESSION_OUTPUT_NAMES:
    print(f"  {target}: {prediction_results[target]:,.2f}")

print("\nClassification Predictions:")
for target in CLASSIFICATION_OUTPUT_NAMES:
    prob = prediction_results[f'{target}_probability']
    pred = prediction_results[f'{target}_prediction']
    print(f"  {target}: {pred} (probability: {prob:.4f})")

## 15. Conclusion

### Summary

This notebook implemented a **Multi-Task Neural Network (MTNN)** for predicting typhoon impacts in the Philippines. The key features include:

1. **Shared Feature Learning**: Common dense layers (128 → 64 → 32) extract shared representations from typhoon characteristics.

2. **Multiple Output Heads**:
   - **6 Regression heads**: Predict continuous values (families, persons, barangays, cost, partially/totally damaged)
   - **3 Classification heads**: Predict binary outcomes (casualties: dead, injured/ill, missing)

3. **Balanced Training**: Custom loss weights help balance regression and classification tasks, with higher weights for rare events.

### Files Generated:
- `../models/mtl_typhoon_prediction_model.h5` - Trained model (H5 format)
- `../models/mtl_typhoon_prediction_model.keras` - Trained model (Keras format)
- `../models/mtl_scalers.joblib` - Feature and target scalers
- `../models/mtl_evaluation_results.joblib` - Evaluation metrics
- `../models/mtl_training_history.png` - Training curves
- `../models/mtl_actual_vs_predicted.png` - Regression scatter plots
- `../models/mtl_confusion_matrices.png` - Classification confusion matrices

### Next Steps:
1. Fine-tune hyperparameters (learning rate, layer sizes, dropout rates)
2. Experiment with different architectures (e.g., residual connections)
3. Add more features (geographic, temporal)
4. Implement class weighting for imbalanced classification targets
5. Deploy the model for real-time predictions